In [1]:
import pandas as pd
import numpy as np

In [2]:
def clean_up(phrase):
    phrase = phrase.lstrip()
    phrase = phrase.rstrip()
    phrase = phrase.upper()
    return phrase

In [3]:
# Import restrictions
rest = pd.read_excel("../../../../Vendors/. Restrictions\All Restricted List.xlsx")
rest.columns = map(str.lower, rest.columns)
rest['isbn13'] = rest['isbn13'].astype(str)
for col in ['isbn13', 'vendor']:
    rest[col] = rest.apply(lambda x: clean_up(x[col]), axis =1)

rest = rest[['isbn13','vendor']]
rest.head()

,isbn13,vendor
0,9780323567428,RISKY
1,9780500292105,RISKY
2,9781936523443,RISKY
3,9780134149530,RISKY
4,9780134205588,RISKY


In [4]:
rest.shape

(89859, 2)

In [5]:
risky = rest[rest['vendor'] == 'RISKY']
risky.shape

(486, 2)

In [6]:
# import suppliers excel
sup_exc = pd.read_csv('suppliers_excel.csv')
sup_exc['isbn'] = sup_exc['isbn'].astype(str)
sup_exc.head()

,isbn,title,author,pub,supplier,currency,rate,price,discount,invoice_fee,lp_fee,shipping cost,max quantity,min,region
0,9780000079145,NaN,NaN,LWW,ABE,GBP,1.237001,67.00,0.55,0.0,0,3.47,10000,0,EUROPE
1,9780000003201,NaN,NaN,LWW,ABE,GBP,1.237001,265.00,0.55,0.0,0,3.47,10000,0,EUROPE
2,9780000002402,NaN,NaN,LWW,ABE,GBP,1.237001,306.00,0.55,0.0,0,3.47,10000,0,EUROPE
3,9780000002679,NaN,NaN,LWW,ABE,GBP,1.237001,425.00,0.55,0.0,0,3.47,10000,0,EUROPE
4,9780000050090,NaN,NaN,LWW,ABE,GBP,1.237001,17.99,0.55,0.0,0,3.47,10000,0,EUROPE


In [7]:
sup_exc = sup_exc[['isbn', 'title', 'author', 'pub', 'supplier','currency','rate','price','discount','invoice_fee','lp_fee','shipping cost']]

In [8]:
sup_sql = pd.read_csv('supplier_sql.csv')
sup_sql['isbn'] = sup_sql['isbn'].astype(str)
sup_sql.head()

,isbn,title,author,pub,supplier,currency,rate,price,discount,invoice_fee,lp_fee,shipping cost
0,9781260241044,NaN,NaN,MCGRAW,ALEK,GBP,1.243139,-1.01,0.47,0.005,0.0,0.5
1,9781260241020,NaN,NaN,MCGRAW,ALEK,GBP,1.243139,-1.01,0.47,0.005,0.0,0.5
2,9781260241044,NaN,NaN,MCGRAW,ALEK,EUR,1.087025,-0.01,0.47,0.005,0.0,0.5
3,9781260241020,NaN,NaN,MCGRAW,ALEK,EUR,1.087025,-0.01,0.47,0.005,0.0,0.5
4,9780786820771,NaN,NaN,HACHETTE LIVE,ALEK,USD,1.000000,0.01,0.59,0.005,0.0,0.5


In [9]:
all_sup = sup_exc.append(sup_sql, ignore_index= True)

In [10]:
all_sup['inv_price_before_exc'] = all_sup['price']*(1-all_sup['discount'])
all_sup['landed_cost'] = all_sup['inv_price_before_exc'] * all_sup['rate'] + all_sup['shipping cost']

In [11]:
all_sup.head()

,isbn,title,author,pub,supplier,currency,rate,price,discount,invoice_fee,lp_fee,shipping cost,inv_price_before_exc,landed_cost
0,9780000079145,NaN,NaN,LWW,ABE,GBP,1.237001,67.00,0.55,0.0,0.0,3.47,30.1500,40.765571
1,9780000003201,NaN,NaN,LWW,ABE,GBP,1.237001,265.00,0.55,0.0,0.0,3.47,119.2500,150.982334
2,9780000002402,NaN,NaN,LWW,ABE,GBP,1.237001,306.00,0.55,0.0,0.0,3.47,137.7000,173.804997
3,9780000002679,NaN,NaN,LWW,ABE,GBP,1.237001,425.00,0.55,0.0,0.0,3.47,191.2500,240.046385
4,9780000050090,NaN,NaN,LWW,ABE,GBP,1.237001,17.99,0.55,0.0,0.0,3.47,8.0955,13.484139


In [12]:
all_sup.shape

(3890304, 14)

In [13]:
all_sup = all_sup.sort_values(by='landed_cost', ascending=True)

In [14]:
all_sup.head()

,isbn,title,author,pub,supplier,currency,rate,price,discount,invoice_fee,lp_fee,shipping cost,inv_price_before_exc,landed_cost
499682,9781260241044,NaN,NaN,MCGRAW,ALEK,GBP,1.243139,-1.01,0.47,0.005,0.0,0.5,-0.5353,-0.165453
499683,9781260241020,NaN,NaN,MCGRAW,ALEK,GBP,1.243139,-1.01,0.47,0.005,0.0,0.5,-0.5353,-0.165453
499685,9781260241020,NaN,NaN,MCGRAW,ALEK,EUR,1.087025,-0.01,0.47,0.005,0.0,0.5,-0.0053,0.494239
499684,9781260241044,NaN,NaN,MCGRAW,ALEK,EUR,1.087025,-0.01,0.47,0.005,0.0,0.5,-0.0053,0.494239
499686,9780786820771,NaN,NaN,HACHETTE LIVE,ALEK,USD,1.000000,0.01,0.59,0.005,0.0,0.5,0.0041,0.504100


In [15]:
all_sup = pd.merge(all_sup, rest, how='left', left_on = ['isbn','supplier'], right_on = ['isbn13', 'vendor'] )
all_sup.head()


,isbn,title,author,pub,supplier,currency,rate,price,discount,invoice_fee,lp_fee,shipping cost,inv_price_before_exc,landed_cost,isbn13,vendor
0,9781260241044,NaN,NaN,MCGRAW,ALEK,GBP,1.243139,-1.01,0.47,0.005,0.0,0.5,-0.5353,-0.165453,NaN,NaN
1,9781260241020,NaN,NaN,MCGRAW,ALEK,GBP,1.243139,-1.01,0.47,0.005,0.0,0.5,-0.5353,-0.165453,NaN,NaN
2,9781260241020,NaN,NaN,MCGRAW,ALEK,EUR,1.087025,-0.01,0.47,0.005,0.0,0.5,-0.0053,0.494239,NaN,NaN
3,9781260241044,NaN,NaN,MCGRAW,ALEK,EUR,1.087025,-0.01,0.47,0.005,0.0,0.5,-0.0053,0.494239,NaN,NaN
4,9780786820771,NaN,NaN,HACHETTE LIVE,ALEK,USD,1.000000,0.01,0.59,0.005,0.0,0.5,0.0041,0.504100,NaN,NaN


In [16]:
all_sup.shape

(3890305, 16)

In [17]:
all_sup.drop(
    all_sup[
        (all_sup['isbn'] == all_sup['isbn13']) & (all_sup['supplier'] == all_sup['vendor'])
        ].index, inplace = True
)

In [18]:
all_sup.shape

(3884669, 16)

In [19]:
all_sup = all_sup[~all_sup['isbn'].isin(list(risky['isbn13']))]

In [20]:
all_sup.shape

(3883035, 16)

In [21]:
all_sup.drop(columns = ['isbn13', 'vendor'], inplace = True)

In [22]:
all_sup.head()

,isbn,title,author,pub,supplier,currency,rate,price,discount,invoice_fee,lp_fee,shipping cost,inv_price_before_exc,landed_cost
0,9781260241044,NaN,NaN,MCGRAW,ALEK,GBP,1.243139,-1.01,0.47,0.005,0.0,0.5,-0.5353,-0.165453
1,9781260241020,NaN,NaN,MCGRAW,ALEK,GBP,1.243139,-1.01,0.47,0.005,0.0,0.5,-0.5353,-0.165453
2,9781260241020,NaN,NaN,MCGRAW,ALEK,EUR,1.087025,-0.01,0.47,0.005,0.0,0.5,-0.0053,0.494239
3,9781260241044,NaN,NaN,MCGRAW,ALEK,EUR,1.087025,-0.01,0.47,0.005,0.0,0.5,-0.0053,0.494239
4,9780786820771,NaN,NaN,HACHETTE LIVE,ALEK,USD,1.000000,0.01,0.59,0.005,0.0,0.5,0.0041,0.504100


In [23]:
all_sup['supplier'].unique()

array(['ALEK', 'PROBOOK', 'RAJAN', 'FELIPE', 'ARMANDO', 'WC', 'BILLSON',
       'BMT', 'SENAD', 'ABE', 'ZOOKAL', 'COINFO', 'UT', 'LAURENTIU',
       'BERTRAMS', 'APEX', 'BRYAN', 'GAUTAM', 'SOHRAB', 'FRIDAY',
       'BILLSON T', 'BILLSON - T', 'BILLSON - SCI', 'TEXTBOOK TRADE HK',
       'MYAMMAR'], dtype=object)

In [24]:
all_sup = all_sup.fillna('N/A')
all_sup.isna().sum()

isbn                    0
title                   0
author                  0
pub                     0
supplier                0
currency                0
rate                    0
price                   0
discount                0
invoice_fee             0
lp_fee                  0
shipping cost           0
inv_price_before_exc    0
landed_cost             0
dtype: int64

In [25]:
# Removing Austalian Suppliers
no_aus = all_sup[~(
    (all_sup['supplier']=='COINFO') |
    (all_sup['supplier']=='ZOOKAL') |
    all_sup['supplier'].str.contains("BILLSON") | 
    (all_sup['supplier']=='APEX') |
    (all_sup['supplier']=='SOHRAB') |
    (all_sup['supplier']=='BRYAN') |
    (all_sup['supplier']=='BERTRAMS') |
    (all_sup['supplier']=='GAUTAM')
    )]
no_aus.shape




(1993447, 14)

In [26]:
#Dropping dupplicates

no_dup = no_aus.drop_duplicates(subset='isbn', keep="first")

In [27]:
no_dup.shape

(898008, 14)

In [28]:
retail_list = []

In [29]:
akademos = []

In [30]:
#Retail:
retail_cost = all_sup[all_sup['isbn'].isin(retail_list)]
retail_cost.to_csv('retail/retail.csv', index= False)

#Akademos:
akademos_csv = no_aus[no_aus['isbn'].isin(akademos)]
akademos_csv.to_csv('retail/akademos.csv', index= False)

#Pearson:
pearson = no_dup[no_dup['pub'].str.contains("PEARSON")]
pearson.to_csv('publishers/pearson.csv', index= False)

#Elsevier
elsevier = no_dup[no_dup['pub'].str.contains("ELSEVIER")]
elsevier.to_csv('publishers/elsevier.csv', index= False)

# CENGAGE
cengage = no_dup[no_dup['pub'].str.contains("CENGAGE")]
cengage.to_csv('publishers/cengage.csv', index = False)

#Sage
sage = no_dup[no_dup['pub'].str.contains("SAGE")]
sage.to_csv('publishers/sage.csv', index = False)

#McGraw
mcgraw = no_dup[no_dup['pub'].str.contains("MCGRAW")]
mcgraw.to_csv('publishers/mcgraw.csv', index = False)

#LLW_WKLUWER
lww_wkluwer = no_dup[no_dup['pub'].str.contains('LWW') | no_dup['pub'].str.contains('W KLUWER')]
lww_wkluwer.to_csv('publishers/lww_wkluwer.csv', index = False)

# NORTON
norton = no_dup[no_dup['pub'].str.contains("NORTON")]
mcgraw.to_csv('publishers/norton.csv', index = False)

#LLW_WKLUWER
macmillan_humankinetics = no_dup[no_dup['pub'].str.contains('MACMILLAN') | no_dup['pub'].str.contains('HUMAN KINETICS')]
macmillan_humankinetics.to_csv('publishers/macmillan_humankinetics.csv', index = False)

#oxford
oxford_fadavis = no_dup[no_dup['pub'].str.contains('OXFORD') | no_dup['pub'].str.contains('FA DAVIS')]
oxford_fadavis.to_csv('publishers/oxford_fadavis.csv', index = False)


# “T&F / CRC / Routledge / Guilford".
t_f= no_dup[no_dup['pub'].str.contains('T&F') | no_dup['pub'].str.contains('ROUTLEDGE') | no_dup['pub'].str.contains('GUILFORD') | (no_dup['pub'] == 'CRC')]
t_f.to_csv('publishers/t&f.csv', index = False)

# J&B
j_b = no_dup[no_dup['pub'].str.contains('J&B')]
j_b.to_csv('publishers/j&b.csv', index = False)

# WILEY / SPRINGER / THIEME
wiley_springer_thieme = no_dup[no_dup['pub'].str.contains('WILEY') | no_dup['pub'].str.contains('SPRINGER') | no_dup['pub'].str.contains('THIEME')]
wiley_springer_thieme.to_csv('publishers/wiley_springer_thieme.csv', index = False)

# REMAINING PUB
remaining = no_dup[
    ~(
        no_dup['pub'].str.contains('WILEY') | no_dup['pub'].str.contains('SPRINGER') |
        no_dup['pub'].str.contains('THIEME') | no_dup['pub'].str.contains("PEARSON") |
        no_dup['pub'].str.contains("ELSEVIER") | no_dup['pub'].str.contains("CENGAGE") |
        no_dup['pub'].str.contains("SAGE") | no_dup['pub'].str.contains("MCGRAW") |
        no_dup['pub'].str.contains('LWW') | no_dup['pub'].str.contains('W KLUWER') |
        no_dup['pub'].str.contains('MACMILLAN') | no_dup['pub'].str.contains('HUMAN KINETICS') |
        no_dup['pub'].str.contains('OXFORD') | no_dup['pub'].str.contains('FA DAVIS') |
        no_dup['pub'].str.contains('T&F') | no_dup['pub'].str.contains('ROUTLEDGE') |
        no_dup['pub'].str.contains('GUILFORD') | (no_dup['pub'] == 'CRC') |
        no_dup['pub'].str.contains('J&B') | no_dup['pub'].str.contains('WILEY') |
        no_dup['pub'].str.contains('SPRINGER') | no_dup['pub'].str.contains('THIEME') |
        no_dup['pub'].str.contains("NORTON")
    )
    ]
remaining.to_csv('publishers/remaining.csv', index = False)

